<a href="https://colab.research.google.com/github/gulliyevn/ML-NLP-LLMProjects/blob/main/TraderPlusOnline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
# ЯЧЕЙКА 1: УСТАНОВКА БИБЛИОТЕК И ИМПОРТЫ
# ==================================================================================

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (LSTM, Dense, Dropout, Input,
                                   BatchNormalization, MultiHeadAttention,
                                   LayerNormalization, Add, GlobalAveragePooling1D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('dark_background')
sns.set_palette("husl")

print("🚀 LSTM FOREX PREDICTION SYSTEM")
print("="*50)
print("✅ Все библиотеки загружены!")
print(f"🔧 TensorFlow версия: {tf.__version__}")
print(f"🔧 GPU доступно: {len(tf.config.experimental.list_physical_devices('GPU'))}")

🚀 LSTM FOREX PREDICTION SYSTEM
✅ Все библиотеки загружены!
🔧 TensorFlow версия: 2.18.0
🔧 GPU доступно: 1


In [93]:
# ЯЧЕЙКА 2: ЗАГРУЗКА ДАННЫХ EUR/USD 15M
# ==================================================================================

def load_forex_data():
    """
    Загрузка EUR/USD данных на 15-минутном таймфрейме
    """
    print("📥 Загружаем EUR/USD данные (15-минутный таймфрейм)...")

    try:
        # Загрузка данных за последние 60 дней (15m интервал)
        data = yf.download(
            'EURUSD=X',
            period='60d',
            interval='15m',
            progress=False,
            auto_adjust=True,
            prepost=False
        )

        if data.empty:
            print("❌ Не удалось загрузить данные")
            return None

        # Очистка колонок
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [col[0] for col in data.columns]

        # Удаляем пропуски
        data = data.dropna()

        # Базовые расчеты
        data['Returns'] = data['Close'].pct_change()
        data['HL_Range'] = (data['High'] - data['Low']) / data['Close']

        print(f"✅ Загружено {len(data):,} баров EUR/USD")
        print(f"📅 Период: {data.index[0].strftime('%Y-%m-%d %H:%M')} - {data.index[-1].strftime('%Y-%m-%d %H:%M')}")
        print(f"💹 Ценовой диапазон: {data['Close'].min():.5f} - {data['Close'].max():.5f}")

        return data

    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        return None

# Загружаем данные
forex_data = load_forex_data()

if forex_data is not None:
    print(f"\n📊 Структура данных:")
    print(forex_data.head())
    print(f"\n📋 Колонки: {list(forex_data.columns)}")
else:
    print("❌ Данные не загружены!")

📥 Загружаем EUR/USD данные (15-минутный таймфрейм)...
✅ Загружено 5,653 баров EUR/USD
📅 Период: 2025-03-26 00:00 - 2025-06-17 15:00
💹 Ценовой диапазон: 1.07377 - 1.16320

📊 Структура данных:
                              Close      High       Low      Open  Volume  \
Datetime                                                                    
2025-03-26 00:00:00+00:00  1.079331  1.079331  1.079098  1.079098       0   
2025-03-26 00:15:00+00:00  1.079331  1.079564  1.079214  1.079331       0   
2025-03-26 00:30:00+00:00  1.078981  1.079447  1.078981  1.079331       0   
2025-03-26 00:45:00+00:00  1.078865  1.078981  1.078749  1.078981       0   
2025-03-26 01:00:00+00:00  1.078516  1.078632  1.078167  1.078516       0   

                            Returns  HL_Range  
Datetime                                       
2025-03-26 00:00:00+00:00       NaN  0.000216  
2025-03-26 00:15:00+00:00  0.000000  0.000324  
2025-03-26 00:30:00+00:00 -0.000324  0.000432  
2025-03-26 00:45:00+00:00 -0.

In [94]:
# ЯЧЕЙКА 3: УЛУЧШЕННЫЕ ТЕХНИЧЕСКИЕ ИНДИКАТОРЫ
# ==================================================================================

def calculate_enhanced_technical_indicators(data):
    """
    Расширенные технические индикаторы на основе документа
    """
    print("🔧 Расчет улучшенных технических индикаторов...")

    df = data.copy()
    high = df['High']
    low = df['Low']
    close = df['Close']
    volume = df.get('Volume', pd.Series(index=df.index, data=1))  # Если нет объема

    # 1. Скользящие средние (улучшенные)
    df['SMA_9'] = close.rolling(window=9).mean()
    df['SMA_21'] = close.rolling(window=21).mean()
    df['EMA_12'] = close.ewm(span=12).mean()
    df['EMA_26'] = close.ewm(span=26).mean()

    # Кроссы MA
    df['MA_Cross'] = np.where(df['SMA_9'] > df['SMA_21'], 1, -1)
    df['EMA_Cross'] = np.where(df['EMA_12'] > df['EMA_26'], 1, -1)

    # 2. RSI (улучшенный)
    delta = close.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # RSI дивергенция и уровни
    df['RSI_Oversold'] = np.where(df['RSI'] < 30, 1, 0)
    df['RSI_Overbought'] = np.where(df['RSI'] > 70, 1, 0)
    df['RSI_Momentum'] = df['RSI'].diff()

    # 3. MACD (улучшенный)
    ema_fast = close.ewm(span=12).mean()
    ema_slow = close.ewm(span=26).mean()
    df['MACD'] = ema_fast - ema_slow
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    df['MACD_Cross'] = np.where(df['MACD'] > df['MACD_Signal'], 1, -1)

    # 4. Bollinger Bands (улучшенные)
    sma_20 = close.rolling(window=20).mean()
    std_20 = close.rolling(window=20).std()
    bb_upper = sma_20 + (2 * std_20)
    bb_lower = sma_20 - (2 * std_20)
    df['BB_Position'] = (close - bb_lower) / (bb_upper - bb_lower)
    df['BB_Width'] = (bb_upper - bb_lower) / sma_20
    df['BB_Squeeze'] = np.where(df['BB_Width'] < df['BB_Width'].rolling(20).mean(), 1, 0)

    # 5. ATR (улучшенный)
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df['ATR'] = true_range.rolling(window=14).mean()
    df['ATR_Percent'] = df['ATR'] / close * 100
    df['ATR_Ratio'] = df['ATR'] / df['ATR'].rolling(50).mean()

    # 6. ADX - Тренд/Флет (ИЗ ДОКУМЕНТА!)
    def calculate_adx(high, low, close, period=14):
        plus_dm = high.diff()
        minus_dm = low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0

        tr = pd.concat([high - low,
                       abs(high - close.shift(1)),
                       abs(low - close.shift(1))], axis=1).max(axis=1)

        atr = tr.rolling(period).mean()
        plus_di = 100 * (plus_dm.rolling(period).mean() / atr)
        minus_di = 100 * (minus_dm.abs().rolling(period).mean() / atr)

        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
        adx = dx.rolling(period).mean()

        return adx, plus_di, minus_di

    df['ADX'], df['DI_Plus'], df['DI_Minus'] = calculate_adx(high, low, close)
    df['ADX_Trend'] = np.where(df['ADX'] > 25, 1, 0)  # 1 = тренд, 0 = флет

    # 7. Williams %R (ИЗ ДОКУМЕНТА!)
    df['Williams_R'] = -100 * (high.rolling(14).max() - close) / (high.rolling(14).max() - low.rolling(14).min())
    df['Williams_Oversold'] = np.where(df['Williams_R'] > -20, 1, 0)
    df['Williams_Overbought'] = np.where(df['Williams_R'] < -80, 1, 0)

    # 8. CCI - Commodity Channel Index (ИЗ ДОКУМЕНТА!)
    typical_price = (high + low + close) / 3
    sma_tp = typical_price.rolling(20).mean()
    mad = typical_price.rolling(20).apply(lambda x: abs(x - x.mean()).mean())
    df['CCI'] = (typical_price - sma_tp) / (0.015 * mad)
    df['CCI_Extreme'] = np.where(abs(df['CCI']) > 100, 1, 0)

    # 9. Stochastic Oscillator (ИЗ ДОКУМЕНТА!)
    low_14 = low.rolling(14).min()
    high_14 = high.rolling(14).max()
    df['Stoch_K'] = 100 * (close - low_14) / (high_14 - low_14)
    df['Stoch_D'] = df['Stoch_K'].rolling(3).mean()
    df['Stoch_Cross'] = np.where(df['Stoch_K'] > df['Stoch_D'], 1, -1)

    # 10. Волатильность и статистика
    df['Volatility_5'] = close.pct_change().rolling(5).std() * 100
    df['Volatility_20'] = close.pct_change().rolling(20).std() * 100
    df['Vol_Ratio'] = df['Volatility_5'] / df['Volatility_20']

    # 11. Momentum индикаторы
    df['ROC_5'] = (close - close.shift(5)) / close.shift(5) * 100
    df['ROC_10'] = (close - close.shift(10)) / close.shift(10) * 100
    df['Momentum'] = close / close.shift(10) * 100

    # 12. Временные признаки (улучшенные)
    df['Hour'] = df.index.hour
    df['DayOfWeek'] = df.index.dayofweek
    df['IsLondonSession'] = np.where((df['Hour'] >= 8) & (df['Hour'] <= 16), 1, 0)
    df['IsNYSession'] = np.where((df['Hour'] >= 13) & (df['Hour'] <= 21), 1, 0)
    df['IsOverlap'] = np.where((df['Hour'] >= 13) & (df['Hour'] <= 16), 1, 0)

    # 13. Лаговые признаки (расширенные)
    for lag in [1, 2, 3, 4, 5]:
        df[f'Returns_lag_{lag}'] = df['Returns'].shift(lag)
        df[f'RSI_lag_{lag}'] = df['RSI'].shift(lag)
        df[f'MACD_lag_{lag}'] = df['MACD'].shift(lag)

    # 14. Взаимодействия индикаторов (НОВОЕ!)
    df['RSI_MACD_Interaction'] = df['RSI'] * df['MACD_Cross']
    df['BB_RSI_Interaction'] = df['BB_Position'] * df['RSI'] / 100
    df['ADX_RSI_Strength'] = df['ADX'] * df['RSI'] / 100
    df['Vol_Momentum'] = df['Vol_Ratio'] * df['ROC_5']

    # 15. Кластерные сигналы
    bullish_signals = (
        (df['RSI'] < 30).astype(int) +
        (df['Williams_R'] < -80).astype(int) +
        (df['MACD_Cross'] == 1).astype(int) +
        (df['MA_Cross'] == 1).astype(int) +
        (df['Stoch_Cross'] == 1).astype(int)
    )
    df['Bullish_Cluster'] = bullish_signals

    bearish_signals = (
        (df['RSI'] > 70).astype(int) +
        (df['Williams_R'] > -20).astype(int) +
        (df['MACD_Cross'] == -1).astype(int) +
        (df['MA_Cross'] == -1).astype(int) +
        (df['Stoch_Cross'] == -1).astype(int)
    )
    df['Bearish_Cluster'] = bearish_signals

    print(f"✅ Рассчитано {len([col for col in df.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']])} индикаторов")
    print(f"📊 Добавлено из документа: ADX, Williams %R, CCI, Stochastic")
    print(f"🔥 Новые фичи: взаимодействия, кластеры, сессии")

    return df

    # Рассчитываем индикаторы
if forex_data is not None:
    forex_with_indicators = calculate_enhanced_technical_indicators(forex_data)  # ← НОВАЯ ФУНКЦИЯ
    print(f"\n📊 Данные с индикаторами: {forex_with_indicators.shape}")
    print(f"📋 Все колонки: {list(forex_with_indicators.columns)}")

🔧 Расчет улучшенных технических индикаторов...
✅ Рассчитано 66 индикаторов
📊 Добавлено из документа: ADX, Williams %R, CCI, Stochastic
🔥 Новые фичи: взаимодействия, кластеры, сессии

📊 Данные с индикаторами: (5653, 71)
📋 Все колонки: ['Close', 'High', 'Low', 'Open', 'Volume', 'Returns', 'HL_Range', 'SMA_9', 'SMA_21', 'EMA_12', 'EMA_26', 'MA_Cross', 'EMA_Cross', 'RSI', 'RSI_Oversold', 'RSI_Overbought', 'RSI_Momentum', 'MACD', 'MACD_Signal', 'MACD_Histogram', 'MACD_Cross', 'BB_Position', 'BB_Width', 'BB_Squeeze', 'ATR', 'ATR_Percent', 'ATR_Ratio', 'ADX', 'DI_Plus', 'DI_Minus', 'ADX_Trend', 'Williams_R', 'Williams_Oversold', 'Williams_Overbought', 'CCI', 'CCI_Extreme', 'Stoch_K', 'Stoch_D', 'Stoch_Cross', 'Volatility_5', 'Volatility_20', 'Vol_Ratio', 'ROC_5', 'ROC_10', 'Momentum', 'Hour', 'DayOfWeek', 'IsLondonSession', 'IsNYSession', 'IsOverlap', 'Returns_lag_1', 'RSI_lag_1', 'MACD_lag_1', 'Returns_lag_2', 'RSI_lag_2', 'MACD_lag_2', 'Returns_lag_3', 'RSI_lag_3', 'MACD_lag_3', 'Returns_la

In [95]:
# ЯЧЕЙКА 4: СОЗДАНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ
# ==================================================================================

def create_target_variable(data, forecast_horizon=4):
    """
    Создание целевой переменной для прогнозирования
    forecast_horizon=4 означает прогноз на 1 час вперед (4 * 15min)
    """
    print(f"🎯 Создание целевой переменной (прогноз на {forecast_horizon * 15} минут)...")

    df = data.copy()

    # Будущие цены
    future_close = df['Close'].shift(-forecast_horizon)
    current_close = df['Close']

    # Расчет изменения цены
    price_change = (future_close - current_close) / current_close

    # Классификация направления
    # Используем 0.5 * ATR как минимальное значимое движение
    min_movement = df['ATR_Percent'] / 100 * 0.5

    target = np.where(
        price_change > min_movement, 1,  # BUY
        np.where(price_change < -min_movement, -1,  # SELL
                0)  # HOLD
    )

    df['Target'] = target
    df['Price_Change'] = price_change

    # Удаляем последние строки без целевой переменной
    df = df[:-forecast_horizon].copy()

    # Статистика
    target_counts = pd.Series(target[~np.isnan(target)]).value_counts().sort_index()
    total = len(target[~np.isnan(target)])

    print(f"📊 Распределение целевой переменной:")
    labels = {-1: 'SELL', 0: 'HOLD', 1: 'BUY'}
    for value, count in target_counts.items():
        if not np.isnan(value):
            label = labels.get(int(value), f'Unknown({value})')
            percentage = count / total * 100
            print(f"   {label}: {count:,} ({percentage:.1f}%)")

    return df

# Создаем целевую переменную
if 'forex_with_indicators' in locals():
    forex_final = create_target_variable(forex_with_indicators)
    print(f"\n📊 Финальные данные: {forex_final.shape}")

🎯 Создание целевой переменной (прогноз на 60 минут)...
📊 Распределение целевой переменной:
   SELL: 1,856 (32.8%)
   HOLD: 1,710 (30.2%)
   BUY: 2,087 (36.9%)

📊 Финальные данные: (5649, 73)


In [96]:
# ЯЧЕЙКА 5: УЛУЧШЕННАЯ ПОДГОТОВКА ПРИЗНАКОВ ДЛЯ LSTM
# ==================================================================================

def prepare_enhanced_features_for_lstm(data):
    """
    Подготовка расширенных признаков для LSTM модели
    """
    print("🔧 Подготовка улучшенных признаков для LSTM...")

    # Основные признаки
    core_features = [
        'Close', 'High', 'Low', 'Open', 'Returns', 'HL_Range'
    ]

    # Технические индикаторы (базовые)
    technical_features = [
        'SMA_9', 'SMA_21', 'EMA_12', 'EMA_26', 'RSI', 'MACD', 'MACD_Signal',
        'MACD_Histogram', 'BB_Position', 'BB_Width', 'ATR_Percent', 'ATR_Ratio'
    ]

    # Новые индикаторы из документа
    advanced_features = [
        'ADX', 'DI_Plus', 'DI_Minus', 'Williams_R', 'CCI', 'Stoch_K', 'Stoch_D',
        'Volatility_5', 'Volatility_20', 'Vol_Ratio', 'ROC_5', 'ROC_10', 'Momentum'
    ]

    # Временные признаки
    time_features = [
        'Hour', 'DayOfWeek', 'IsLondonSession', 'IsNYSession', 'IsOverlap'
    ]

    # Кроссы и сигналы
    signal_features = [
        'MA_Cross', 'EMA_Cross', 'MACD_Cross', 'Stoch_Cross', 'ADX_Trend',
        'RSI_Oversold', 'RSI_Overbought', 'Williams_Oversold', 'Williams_Overbought',
        'CCI_Extreme', 'BB_Squeeze'
    ]

    # Лаговые признаки
    lag_features = [
        'Returns_lag_1', 'Returns_lag_2', 'Returns_lag_3', 'Returns_lag_4', 'Returns_lag_5',
        'RSI_lag_1', 'RSI_lag_2', 'RSI_lag_3', 'MACD_lag_1', 'MACD_lag_2'
    ]

    # Взаимодействия
    interaction_features = [
        'RSI_MACD_Interaction', 'BB_RSI_Interaction', 'ADX_RSI_Strength', 'Vol_Momentum'
    ]

    # Кластеры
    cluster_features = [
        'Bullish_Cluster', 'Bearish_Cluster'
    ]

    # Объединяем все признаки
    all_features = (core_features + technical_features + advanced_features +
                   time_features + signal_features + lag_features +
                   interaction_features + cluster_features)

    # Фильтруем только существующие признаки
    available_features = [col for col in all_features if col in data.columns]

    # Создаем матрицу признаков
    X = data[available_features].copy()
    y = data['Target'].values

    # Удаляем строки с NaN
    mask = ~(np.isnan(y) | X.isnull().any(axis=1))
    X_clean = X[mask].fillna(method='ffill').fillna(0)
    y_clean = y[mask]

    print(f"✅ Подготовлено признаков: {len(available_features)} (было 19)")
    print(f"📊 Чистых образцов: {len(X_clean):,}")

    # Разбивка по категориям
    feature_categories = {
        'Основные': [f for f in core_features if f in available_features],
        'Технические': [f for f in technical_features if f in available_features],
        'Продвинутые': [f for f in advanced_features if f in available_features],
        'Временные': [f for f in time_features if f in available_features],
        'Сигналы': [f for f in signal_features if f in available_features],
        'Лаги': [f for f in lag_features if f in available_features],
        'Взаимодействия': [f for f in interaction_features if f in available_features],
        'Кластеры': [f for f in cluster_features if f in available_features]
    }

    print(f"\n📋 Признаки по категориям:")
    for category, features in feature_categories.items():
        if features:
            print(f"   {category}: {len(features)} ({', '.join(features[:3])}{'...' if len(features) > 3 else ''})")

    # Статистика классов
    unique, counts = np.unique(y_clean, return_counts=True)
    print(f"\n📊 Итоговое распределение:")
    labels = {-1: 'SELL', 0: 'HOLD', 1: 'BUY'}
    for val, count in zip(unique, counts):
        if not np.isnan(val):
            print(f"   {labels.get(int(val))}: {count:,} ({count/len(y_clean)*100:.1f}%)")

    return X_clean, y_clean, available_features, feature_categories

    # Подготавливаем признаки
if 'forex_final' in locals():
    X, y, features, categories = prepare_enhanced_features_for_lstm(forex_final)
    print(f"\n✅ Данные готовы для LSTM: X{X.shape}, y{y.shape}")
else:
    print("⚠️ forex_final не найден. Выполните ячейку №4 сначала.")

🔧 Подготовка улучшенных признаков для LSTM...
✅ Подготовлено признаков: 63 (было 19)
📊 Чистых образцов: 5,587

📋 Признаки по категориям:
   Основные: 6 (Close, High, Low...)
   Технические: 12 (SMA_9, SMA_21, EMA_12...)
   Продвинутые: 13 (ADX, DI_Plus, DI_Minus...)
   Временные: 5 (Hour, DayOfWeek, IsLondonSession...)
   Сигналы: 11 (MA_Cross, EMA_Cross, MACD_Cross...)
   Лаги: 10 (Returns_lag_1, Returns_lag_2, Returns_lag_3...)
   Взаимодействия: 4 (RSI_MACD_Interaction, BB_RSI_Interaction, ADX_RSI_Strength...)
   Кластеры: 2 (Bullish_Cluster, Bearish_Cluster)

📊 Итоговое распределение:
   SELL: 1,837 (32.9%)
   HOLD: 1,681 (30.1%)
   BUY: 2,069 (37.0%)

✅ Данные готовы для LSTM: X(5587, 63), y(5587,)


In [97]:
# ЯЧЕЙКА 6: УЛУЧШЕННАЯ LSTM МОДЕЛЬ С ATTENTION (НОВАЯ ВЕРСИЯ)
# ==================================================================================

import pickle
import os

class EnhancedForexLSTMModel:
    """
    Улучшенная LSTM модель для прогнозирования forex с расширенными возможностями
    Основано на рекомендациях из профессионального документа
    """

    def __init__(self):
        self.model = None
        self.scaler = None
        self.history = None
        self.attention_weights = None

        # УЛУЧШЕННЫЕ ПАРАМЕТРЫ МОДЕЛИ (из документа)
        self.sequence_length = 96  # 24 часа истории (96 * 15min) - как рекомендует документ
        self.lstm_units = [128, 64, 32]  # 3 слоя для лучшего обучения
        self.dropout_rate = 0.3  # Больше dropout против переобучения
        self.recurrent_dropout = 0.2  # Dropout для LSTM связей
        self.attention_heads = 8  # Больше голов attention для лучшего понимания
        self.learning_rate = 0.0001  # Меньше LR для стабильности
        self.batch_size = 32  # Меньше batch для лучшего обучения
        self.epochs = 100  # Больше эпох

        # Регуляризация
        self.l2_reg = 0.001
        self.patience = 15  # Больше терпения для early stopping

        # Новые параметры
        self.use_class_weights = True
        self.save_attention = True

    def create_sequences(self, X, y):
        """Создание последовательностей для LSTM с проверками"""
        print(f"🔧 Создание последовательностей длиной {self.sequence_length}...")

        if len(X) < self.sequence_length:
            print(f"❌ Недостаточно данных! Нужно минимум {self.sequence_length}, есть {len(X)}")
            return None, None

        X_sequences = []
        y_sequences = []

        for i in range(self.sequence_length, len(X)):
            X_sequences.append(X[i-self.sequence_length:i])
            y_sequences.append(y[i])

        X_sequences = np.array(X_sequences, dtype=np.float32)
        y_sequences = np.array(y_sequences)

        print(f"✅ Создано {len(X_sequences):,} последовательностей")
        print(f"📊 Форма X: {X_sequences.shape}, форма y: {y_sequences.shape}")

        return X_sequences, y_sequences

    def prepare_data(self, X, y):
        """Подготовка данных для обучения с расширенной статистикой"""
        print("🔧 Подготовка данных для обучения...")

        # Нормализация с сохранением scaler
        print("📊 Нормализация признаков...")
        self.scaler = RobustScaler()
        X_scaled = self.scaler.fit_transform(X)

        # Сохранение scaler
        os.makedirs('models', exist_ok=True)
        with open('models/scaler.pkl', 'wb') as f:
            pickle.dump(self.scaler, f)
        print("💾 Scaler сохранен в models/scaler.pkl")

        # Создание последовательностей
        X_seq, y_seq = self.create_sequences(X_scaled, y)

        if X_seq is None:
            return None, None, None

        # Разделение данных по времени (важно для временных рядов)
        total_samples = len(X_seq)
        test_size = int(total_samples * 0.15)  # 15% на тест
        val_size = int(total_samples * 0.15)   # 15% на валидацию
        train_size = total_samples - test_size - val_size

        # Временное разделение (последние данные для теста)
        X_train = X_seq[:train_size]
        y_train = y_seq[:train_size]
        X_val = X_seq[train_size:train_size + val_size]
        y_val = y_seq[train_size:train_size + val_size]
        X_test = X_seq[train_size + val_size:]
        y_test = y_seq[train_size + val_size:]

        # Конвертация в категории (SELL=-1, HOLD=0, BUY=1 -> 0,1,2)
        def to_categorical_custom(y_data):
            y_cat = y_data + 1  # [-1,0,1] -> [0,1,2]
            return tf.keras.utils.to_categorical(y_cat, num_classes=3)

        y_train_cat = to_categorical_custom(y_train)
        y_val_cat = to_categorical_custom(y_val)
        y_test_cat = to_categorical_custom(y_test)

        print(f"📊 Разделение данных:")
        print(f"   Train: {len(X_train):,} ({len(X_train)/total_samples*100:.1f}%)")
        print(f"   Val: {len(X_val):,} ({len(X_val)/total_samples*100:.1f}%)")
        print(f"   Test: {len(X_test):,} ({len(X_test)/total_samples*100:.1f}%)")

        # Статистика классов в каждом наборе
        def print_class_distribution(y_data, dataset_name):
            y_integers = np.argmax(y_data, axis=1)
            unique, counts = np.unique(y_integers, return_counts=True)
            class_names = ['SELL', 'HOLD', 'BUY']
            print(f"   {dataset_name}:")
            for i, (cls, count) in enumerate(zip(unique, counts)):
                print(f"     {class_names[cls]}: {count:,} ({count/len(y_data)*100:.1f}%)")

        print_class_distribution(y_train_cat, "Train распределение")
        print_class_distribution(y_val_cat, "Val распределение")
        print_class_distribution(y_test_cat, "Test распределение")

        return (X_train, y_train_cat), (X_val, y_val_cat), (X_test, y_test_cat)

    def build_model(self, input_shape):
        """Создание улучшенной LSTM модели с продвинутой архитектурой"""
        print("🏗️ Создание улучшенной LSTM модели...")
        print(f"📊 Входная форма: {input_shape}")
        print(f"🔧 Параметры: LSTM{self.lstm_units}, Attention heads={self.attention_heads}")

        # Входной слой
        inputs = Input(shape=input_shape)

        # 1-й LSTM слой
        x = LSTM(self.lstm_units[0],
                return_sequences=True,
                dropout=self.dropout_rate,
                recurrent_dropout=self.recurrent_dropout,
                kernel_regularizer=l2(self.l2_reg))(inputs)
        x = LayerNormalization()(x)

        # 2-й LSTM слой
        x = LSTM(self.lstm_units[1],
                return_sequences=True,
                dropout=self.dropout_rate,
                recurrent_dropout=self.recurrent_dropout,
                kernel_regularizer=l2(self.l2_reg))(x)
        x = LayerNormalization()(x)

        # 3-й LSTM слой
        x = LSTM(self.lstm_units[2],
                return_sequences=True,
                dropout=self.dropout_rate,
                recurrent_dropout=self.recurrent_dropout,
                kernel_regularizer=l2(self.l2_reg))(x)
        x = LayerNormalization()(x)

        # Первый Multi-Head Attention слой
        attention_output1 = MultiHeadAttention(
            num_heads=self.attention_heads,
            key_dim=64,
            dropout=0.1
        )(x, x)

        # Residual connection + LayerNorm
        x = Add()([x, attention_output1])
        x = LayerNormalization()(x)

        # Второй Attention слой (меньше голов)
        attention_output2 = MultiHeadAttention(
            num_heads=4,
            key_dim=32,
            dropout=0.1
        )(x, x)

        # Еще одна residual connection
        x = Add()([x, attention_output2])
        x = LayerNormalization()(x)

        # Global Average Pooling для агрегации
        x = GlobalAveragePooling1D()(x)

        # Dense слои с регуляризацией
        x = Dense(64, activation='relu', kernel_regularizer=l2(self.l2_reg))(x)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout_rate)(x)

        x = Dense(32, activation='relu', kernel_regularizer=l2(self.l2_reg))(x)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout_rate)(x)

        x = Dense(16, activation='relu', kernel_regularizer=l2(self.l2_reg))(x)
        x = Dropout(self.dropout_rate)(x)

        # Выходной слой
        outputs = Dense(3, activation='softmax', name='predictions')(x)

        # Создание модели
        model = Model(inputs, outputs)

        # Кастомные метрики F1, Precision, Recall
        def precision_class(class_id):
            def precision(y_true, y_pred):
                y_true_class = tf.cast(tf.equal(tf.argmax(y_true, axis=1), class_id), tf.float32)
                y_pred_class = tf.cast(tf.equal(tf.argmax(y_pred, axis=1), class_id), tf.float32)

                tp = tf.reduce_sum(y_true_class * y_pred_class)
                fp = tf.reduce_sum((1 - y_true_class) * y_pred_class)

                return tp / (tp + fp + tf.keras.backend.epsilon())
            return precision

        def recall_class(class_id):
            def recall(y_true, y_pred):
                y_true_class = tf.cast(tf.equal(tf.argmax(y_true, axis=1), class_id), tf.float32)
                y_pred_class = tf.cast(tf.equal(tf.argmax(y_pred, axis=1), class_id), tf.float32)

                tp = tf.reduce_sum(y_true_class * y_pred_class)
                fn = tf.reduce_sum(y_true_class * (1 - y_pred_class))

                return tp / (tp + fn + tf.keras.backend.epsilon())
            return recall

        def f1_macro(y_true, y_pred):
            def f1_score_class(class_id):
                y_true_class = tf.cast(tf.equal(tf.argmax(y_true, axis=1), class_id), tf.float32)
                y_pred_class = tf.cast(tf.equal(tf.argmax(y_pred, axis=1), class_id), tf.float32)

                tp = tf.reduce_sum(y_true_class * y_pred_class)
                fp = tf.reduce_sum((1 - y_true_class) * y_pred_class)
                fn = tf.reduce_sum(y_true_class * (1 - y_pred_class))

                precision = tp / (tp + fp + tf.keras.backend.epsilon())
                recall = tp / (tp + fn + tf.keras.backend.epsilon())

                return 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())

            f1_0 = f1_score_class(0)  # SELL
            f1_1 = f1_score_class(1)  # HOLD
            f1_2 = f1_score_class(2)  # BUY

            return (f1_0 + f1_1 + f1_2) / 3

        # Компиляция с улучшенным оптимизатором
        model.compile(
            optimizer=Adam(
                learning_rate=self.learning_rate,
                clipnorm=1.0,  # Gradient clipping
                beta_1=0.9,
                beta_2=0.999
            ),
            loss='categorical_crossentropy',
            metrics=[
                'accuracy',
                f1_macro,
                precision_class(0),  # SELL precision
                precision_class(1),  # HOLD precision
                precision_class(2),  # BUY precision
                recall_class(0),     # SELL recall
                recall_class(1),     # HOLD recall
                recall_class(2)      # BUY recall
            ]
        )

        print(f"✅ Улучшенная модель создана!")
        print(f"📊 Параметров: {model.count_params():,}")
        print(f"🔧 Архитектура: {len(self.lstm_units)} LSTM слоя + 2 Attention слоя")
        print(f"📈 Метрики: Accuracy, F1-macro, Precision/Recall по классам")

        return model

    def train(self, train_data, val_data):
        """Обучение модели с расширенными callback'ами"""
        X_train, y_train = train_data
        X_val, y_val = val_data

        print("🚀 Начинаем обучение улучшенной модели...")
        print(f"⏱️ Ожидайте 20-30 минут (сложная архитектура)")

        # Создаем модель
        self.model = self.build_model((X_train.shape[1], X_train.shape[2]))

        # Расширенные callbacks
        callbacks = [
            EarlyStopping(
                monitor='val_f1_macro',
                patience=self.patience,
                restore_best_weights=True,
                verbose=1,
                mode='max'
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=7,
                verbose=1,
                min_lr=1e-7
            ),
            ModelCheckpoint(
                'models/best_enhanced_model.h5',
                monitor='val_f1_macro',
                save_best_only=True,
                verbose=1,
                mode='max'
            )
        ]

        # Вычисление весов классов
        if self.use_class_weights:
            y_integers = np.argmax(y_train, axis=1)
            class_counts = np.bincount(y_integers)
            total_samples = len(y_train)
            class_weights = total_samples / (len(class_counts) * class_counts)
            class_weight_dict = {i: class_weights[i] for i in range(len(class_counts))}

            print(f"⚖️ Веса классов для балансировки:")
            class_names = ['SELL', 'HOLD', 'BUY']
            for i, weight in class_weight_dict.items():
                print(f"   {class_names[i]}: {weight:.3f}")
        else:
            class_weight_dict = None

        # Обучение
        print(f"\n🔥 Запуск обучения с параметрами:")
        print(f"   📏 Sequence length: {self.sequence_length}")
        print(f"   🧠 LSTM архитектура: {self.lstm_units}")
        print(f"   👁️ Attention heads: {self.attention_heads}")
        print(f"   📚 Batch size: {self.batch_size}")
        print(f"   🔄 Max epochs: {self.epochs}")
        print(f"   🎯 Learning rate: {self.learning_rate}")

        self.history = self.model.fit(
            X_train, y_train,
            batch_size=self.batch_size,
            epochs=self.epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            class_weight=class_weight_dict,
            verbose=1,
            shuffle=False  # Критично для временных рядов!
        )

        print("✅ Обучение завершено!")
        return self.history

    def evaluate(self, test_data):
        """Расширенная оценка модели"""
        X_test, y_test = test_data

        print("📊 Расширенная оценка модели...")

        # Предсказания
        y_pred_proba = self.model.predict(X_test, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)
        y_true = np.argmax(y_test, axis=1)

        # Основные метрики
        test_results = self.model.evaluate(X_test, y_test, verbose=0)
        metric_names = self.model.metrics_names

        print(f"🎯 Результаты на тестовых данных:")
        for name, value in zip(metric_names, test_results):
            if 'accuracy' in name:
                print(f"   {name}: {value:.4f} ({value*100:.2f}%)")
            else:
                print(f"   {name}: {value:.4f}")

        # Детальный отчет по классам
        class_names = ['SELL', 'HOLD', 'BUY']
        print(f"\n📋 Детальный отчет по классам:")
        print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

        # Анализ уверенности модели
        confidence = np.max(y_pred_proba, axis=1)
        print(f"\n🎯 Анализ уверенности модели:")
        print(f"   Средняя уверенность: {confidence.mean():.3f}")
        print(f"   Медианная уверенность: {np.median(confidence):.3f}")
        print(f"   Высокая уверенность (>0.8): {(confidence > 0.8).sum()/len(confidence)*100:.1f}%")
        print(f"   Низкая уверенность (<0.5): {(confidence < 0.5).sum()/len(confidence)*100:.1f}%")

        return {
            'accuracy': test_results[1],  # accuracy метрика
            'f1_macro': test_results[2],  # f1_macro метрика
            'loss': test_results[0],
            'predictions': y_pred_proba,
            'predictions_class': y_pred,
            'true_class': y_true,
            'confidence': confidence,
            'confusion_matrix': confusion_matrix(y_true, y_pred),
            'classification_report': classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        }

    def save_model(self, filepath='models/enhanced_forex_model'):
        """Сохранение полной модели"""
        if self.model is not None:
            self.model.save(f"{filepath}.h5")
            print(f"💾 Модель сохранена: {filepath}.h5")

    def load_model(self, filepath='models/enhanced_forex_model.h5'):
        """Загрузка модели"""
        if os.path.exists(filepath):
            self.model = tf.keras.models.load_model(filepath)
            print(f"📂 Модель загружена: {filepath}")
        else:
            print(f"❌ Файл модели не найден: {filepath}")

print("✅ Класс EnhancedForexLSTMModel готов!")
print("🔥 Улучшения:")
print("   • Sequence length увеличен до 96 (24 часа)")
print("   • 3 LSTM слоя + 2 Attention слоя")
print("   • F1-score, Precision, Recall метрики")
print("   • Сохранение scaler и модели")
print("   • Лучшая регуляризация и class weights")
print("   • Gradient clipping и улучшенный Adam")

✅ Класс EnhancedForexLSTMModel готов!
🔥 Улучшения:
   • Sequence length увеличен до 96 (24 часа)
   • 3 LSTM слоя + 2 Attention слоя
   • F1-score, Precision, Recall метрики
   • Сохранение scaler и модели
   • Лучшая регуляризация и class weights
   • Gradient clipping и улучшенный Adam


In [98]:
# # ЯЧЕЙКА 7: ОБУЧЕНИЕ УЛУЧШЕННОЙ МОДЕЛИ (НОВАЯ ВЕРСИЯ)
# # ==================================================================================

# def train_enhanced_forex_model_v2(data):
#     """
#     Обучение новой улучшенной модели EnhancedForexLSTMModel
#     """
#     print("🚀 ЗАПУСК ОБУЧЕНИЯ ENHANCED LSTM МОДЕЛИ V2")
#     print("="*70)

#     # 1. Рассчитываем улучшенные индикаторы
#     print("🔧 Этап 1: Расчет улучшенных индикаторов...")
#     enhanced_data = calculate_enhanced_technical_indicators(data)

#     # 2. Создаем целевую переменную
#     print("🎯 Этап 2: Создание целевой переменной...")
#     final_data = create_target_variable(enhanced_data)

#     # 3. Подготавливаем улучшенные признаки
#     print("📊 Этап 3: Подготовка улучшенных признаков...")
#     X, y, features, categories = prepare_enhanced_features_for_lstm(final_data)

#     print(f"\n📈 СВОДКА ПО ДАННЫМ:")
#     print(f"   📊 Образцов: {len(X):,}")
#     print(f"   🔢 Признаков: {len(features)} (было 19 → прирост +{len(features) - 19})")
#     print(f"   ⏱️ Период: {final_data.index[0].strftime('%Y-%m-%d')} - {final_data.index[-1].strftime('%Y-%m-%d')}")

#     # Статистика по категориям
#     print(f"\n🗂️ ПРИЗНАКИ ПО КАТЕГОРИЯМ:")
#     total_features = 0
#     for category, cat_features in categories.items():
#         if cat_features:
#             count = len(cat_features)
#             total_features += count
#             print(f"   📁 {category}: {count}")
#             # Показываем первые 3 признака в категории
#             sample_features = ', '.join(cat_features[:3])
#             if len(cat_features) > 3:
#                 sample_features += f", ... (+{len(cat_features)-3} еще)"
#             print(f"      └─ {sample_features}")

#     print(f"   📊 Всего: {total_features} признаков")

#     # 4. Создаем и обучаем улучшенную модель
#     print(f"\n🤖 Этап 4: Создание EnhancedForexLSTMModel...")
#     model = EnhancedForexLSTMModel()  # НОВЫЙ КЛАСС!

#     # Показываем параметры модели
#     print(f"\n⚙️ ПАРАМЕТРЫ МОДЕЛИ:")
#     print(f"   📏 Sequence length: {model.sequence_length} баров ({model.sequence_length * 15} минут)")
#     print(f"   🧠 LSTM архитектура: {model.lstm_units}")
#     print(f"   👁️ Attention heads: {model.attention_heads}")
#     print(f"   📚 Batch size: {model.batch_size}")
#     print(f"   🔄 Max epochs: {model.epochs}")
#     print(f"   🎯 Learning rate: {model.learning_rate}")
#     print(f"   📉 Dropout: {model.dropout_rate}")
#     print(f"   ⚖️ L2 regularization: {model.l2_reg}")

#     # Подготавливаем данные
#     data_result = model.prepare_data(X.values, y)
#     if data_result[0] is None:
#         print("❌ Ошибка подготовки данных!")
#         return None, None

#     train_data, val_data, test_data = data_result

#     print(f"\n🔥 Этап 5: ЗАПУСК ОБУЧЕНИЯ...")
#     print(f"⚠️ ВНИМАНИЕ: Обучение займет 20-30 минут!")
#     print(f"📊 Ожидается значительное улучшение качества")
#     print("🔄 Мониторим F1-macro score как основную метрику")

#     # Обучаем модель
#     history = model.train(train_data, val_data)

#     # Оцениваем модель
#     print(f"\n📊 Этап 6: Оценка модели...")
#     results = model.evaluate(test_data)

#     # Сохраняем модель
#     print(f"\n💾 Этап 7: Сохранение модели...")
#     model.save_model()

#     # 8. Создаем расширенную визуализацию
#     print(f"\n📈 Этап 8: Создание расширенной визуализации...")
#     create_enhanced_visualization(history, results, categories, features)

#     # 9. Финальная сводка
#     print_final_summary(results, len(features), model)

#     return model, results, categories, features

# def create_enhanced_visualization(history, results, categories, features):
#     """Создание расширенной визуализации результатов"""

#     plt.figure(figsize=(24, 16))

#     # 1. Training Loss
#     plt.subplot(3, 4, 1)
#     plt.plot(history.history['loss'], label='Train Loss', linewidth=2, color='red')
#     plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2, color='orange')
#     plt.title('📉 Training Loss', fontsize=12, fontweight='bold')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.grid(True, alpha=0.3)

#     # 2. Accuracy
#     plt.subplot(3, 4, 2)
#     plt.plot(history.history['accuracy'], label='Train Acc', linewidth=2, color='blue')
#     plt.plot(history.history['val_accuracy'], label='Val Acc', linewidth=2, color='cyan')
#     plt.title('📈 Accuracy', fontsize=12, fontweight='bold')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.legend()
#     plt.grid(True, alpha=0.3)

#     # 3. F1 Score (главная метрика)
#     plt.subplot(3, 4, 3)
#     plt.plot(history.history['f1_macro'], label='Train F1', linewidth=2, color='green')
#     plt.plot(history.history['val_f1_macro'], label='Val F1', linewidth=2, color='lime')
#     plt.title('🎯 F1-Macro Score', fontsize=12, fontweight='bold')
#     plt.xlabel('Epoch')
#     plt.ylabel('F1 Score')
#     plt.legend()
#     plt.grid(True, alpha=0.3)

#     # 4. Learning Rate
#     plt.subplot(3, 4, 4)
#     if 'lr' in history.history:
#         plt.plot(history.history['lr'], linewidth=2, color='purple')
#         plt.title('📊 Learning Rate', fontsize=12, fontweight='bold')
#         plt.xlabel('Epoch')
#         plt.ylabel('LR')
#         plt.yscale('log')
#         plt.grid(True, alpha=0.3)

#     # 5. Confusion Matrix
#     plt.subplot(3, 4, 5)
#     cm = results['confusion_matrix']
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['SELL', 'HOLD', 'BUY'],
#                 yticklabels=['SELL', 'HOLD', 'BUY'],
#                 cbar_kws={'label': 'Count'})
#     plt.title('🎯 Confusion Matrix', fontsize=12, fontweight='bold')

#     # 6. Class-wise Precision
#     plt.subplot(3, 4, 6)
#     class_names = ['SELL', 'HOLD', 'BUY']
#     precision_scores = [
#         results['classification_report']['SELL']['precision'],
#         results['classification_report']['HOLD']['precision'],
#         results['classification_report']['BUY']['precision']
#     ]
#     bars = plt.bar(class_names, precision_scores, color=['red', 'gray', 'green'], alpha=0.7)
#     plt.title('📊 Precision by Class', fontsize=12, fontweight='bold')
#     plt.ylabel('Precision')
#     plt.ylim(0, 1)
#     # Добавляем значения на столбцы
#     for bar, score in zip(bars, precision_scores):
#         plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
#                 f'{score:.3f}', ha='center', va='bottom')
#     plt.grid(True, alpha=0.3)

#     # 7. Class-wise Recall
#     plt.subplot(3, 4, 7)
#     recall_scores = [
#         results['classification_report']['SELL']['recall'],
#         results['classification_report']['HOLD']['recall'],
#         results['classification_report']['BUY']['recall']
#     ]
#     bars = plt.bar(class_names, recall_scores, color=['red', 'gray', 'green'], alpha=0.7)
#     plt.title('📊 Recall by Class', fontsize=12, fontweight='bold')
#     plt.ylabel('Recall')
#     plt.ylim(0, 1)
#     for bar, score in zip(bars, recall_scores):
#         plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
#                 f'{score:.3f}', ha='center', va='bottom')
#     plt.grid(True, alpha=0.3)

#     # 8. Feature Categories
#     plt.subplot(3, 4, 8)
#     category_counts = {k: len(v) for k, v in categories.items() if v}
#     categories_short = [k[:8] for k in category_counts.keys()]  # Сокращаем названия
#     plt.pie(category_counts.values(), labels=categories_short, autopct='%1.0f%%', startangle=90)
#     plt.title('🗂️ Features by Category', fontsize=12, fontweight='bold')

#     # 9. Predictions Distribution
#     plt.subplot(3, 4, 9)
#     pred_classes = results['predictions_class']
#     unique, counts = np.unique(pred_classes, return_counts=True)
#     pred_counts = [counts[unique == i][0] if i in unique else 0 for i in range(3)]
#     plt.pie(pred_counts, labels=class_names, autopct='%1.1f%%',
#             colors=['red', 'gray', 'green'], startangle=90)
#     plt.title('🎯 Predictions Distribution', fontsize=12, fontweight='bold')

#     # 10. Model Confidence
#     plt.subplot(3, 4, 10)
#     confidence = results['confidence']
#     plt.hist(confidence, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
#     plt.axvline(confidence.mean(), color='red', linestyle='--',
#                 label=f'Mean: {confidence.mean():.3f}')
#     plt.title('📊 Model Confidence', fontsize=12, fontweight='bold')
#     plt.xlabel('Max Probability')
#     plt.ylabel('Count')
#     plt.legend()
#     plt.grid(True, alpha=0.3)

#     # 11. Class Performance Comparison
#     plt.subplot(3, 4, 11)
#     x = np.arange(len(class_names))
#     width = 0.35

#     plt.bar(x - width/2, precision_scores, width, label='Precision', alpha=0.7, color='blue')
#     plt.bar(x + width/2, recall_scores, width, label='Recall', alpha=0.7, color='orange')

#     plt.xlabel('Classes')
#     plt.ylabel('Score')
#     plt.title('⚖️ Precision vs Recall', fontsize=12, fontweight='bold')
#     plt.xticks(x, class_names)
#     plt.legend()
#     plt.grid(True, alpha=0.3)

#     # 12. Training Progress Summary
#     plt.subplot(3, 4, 12)
#     epochs = range(1, len(history.history['loss']) + 1)
#     plt.plot(epochs, history.history['accuracy'], 'b-', label='Train Acc', alpha=0.7)
#     plt.plot(epochs, history.history['val_accuracy'], 'r-', label='Val Acc', alpha=0.7)
#     plt.plot(epochs, history.history['f1_macro'], 'g-', label='Train F1', alpha=0.7)
#     plt.plot(epochs, history.history['val_f1_macro'], 'm-', label='Val F1', alpha=0.7)

#     plt.title('📈 Training Summary', fontsize=12, fontweight='bold')
#     plt.xlabel('Epoch')
#     plt.ylabel('Score')
#     plt.legend()
#     plt.grid(True, alpha=0.3)

#     plt.tight_layout()
#     plt.show()

# def print_final_summary(results, num_features, model):
#     """Печать финальной сводки результатов"""

#     print("\n" + "="*70)
#     print("🎯 ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ ENHANCED LSTM MODEL")
#     print("="*70)

#     # Основные метрики
#     print(f"📊 ОСНОВНЫЕ МЕТРИКИ:")
#     print(f"   🎯 Accuracy: {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
#     print(f"   🏆 F1-Macro: {results['f1_macro']:.4f} ({results['f1_macro']*100:.2f}%)")
#     print(f"   📉 Loss: {results['loss']:.4f}")

#     # Сравнение с предыдущей версией
#     print(f"\n📈 СРАВНЕНИЕ С ПРЕДЫДУЩЕЙ ВЕРСИЕЙ:")
#     print(f"   🔢 Признаков: {num_features} (было 19, прирост +{num_features-19})")
#     print(f"   📏 Sequence length: {model.sequence_length} (было 64)")
#     print(f"   🧠 LSTM слои: {len(model.lstm_units)} (было 2)")
#     print(f"   👁️ Attention heads: {model.attention_heads} (было 4)")

#     # Детальные метрики по классам
#     print(f"\n📋 ДЕТАЛЬНЫЕ МЕТРИКИ ПО КЛАССАМ:")
#     class_names = ['SELL', 'HOLD', 'BUY']
#     for class_name in class_names:
#         if class_name in results['classification_report']:
#             report = results['classification_report'][class_name]
#             print(f"   {class_name}:")
#             print(f"     Precision: {report['precision']:.4f}")
#             print(f"     Recall: {report['recall']:.4f}")
#             print(f"     F1-score: {report['f1-score']:.4f}")
#             print(f"     Support: {report['support']}")

#     # Анализ уверенности
#     confidence = results['confidence']
#     print(f"\n🎯 АНАЛИЗ УВЕРЕННОСТИ МОДЕЛИ:")
#     print(f"   📊 Средняя уверенность: {confidence.mean():.3f}")
#     print(f"   📊 Медианная уверенность: {np.median(confidence):.3f}")
#     print(f"   ✅ Высокая уверенность (>0.8): {(confidence > 0.8).sum()}/{len(confidence)} ({(confidence > 0.8).sum()/len(confidence)*100:.1f}%)")
#     print(f"   ⚠️ Низкая уверенность (<0.5): {(confidence < 0.5).sum()}/{len(confidence)} ({(confidence < 0.5).sum()/len(confidence)*100:.1f}%)")

#     # Практические рекомендации
#     print(f"\n💡 ПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ:")
#     if results['accuracy'] > 0.45:
#         print(f"   ✅ Отличный результат! Модель готова к тестированию")
#     elif results['accuracy'] > 0.40:
#         print(f"   📈 Хороший результат! Можно улучшить еще немного")
#     else:
#         print(f"   ⚠️ Результат ниже ожидаемого. Рекомендуется:")
#         print(f"     • Добавить больше данных")
#         print(f"     • Попробовать другие гиперпараметры")
#         print(f"     • Провести feature engineering")

#     print(f"\n💾 СОХРАНЕНИЕ:")
#     print(f"   📁 Модель: models/enhanced_forex_model.h5")
#     print(f"   📁 Scaler: models/scaler.pkl")
#     print(f"   📁 Лучшая модель: models/best_enhanced_model.h5")

#     print("="*70)
#     print("✅ ОБУЧЕНИЕ ENHANCED МОДЕЛИ ЗАВЕРШЕНО!")
#     print("="*70)

# # ОСНОВНОЙ ЗАПУСК
# if 'forex_data' in locals():
#     print("🔥 ВСЕ ГОТОВО! ЗАПУСКАЕМ ENHANCED ОБУЧЕНИЕ V2...")
#     print("⏱️ Ожидайте 25-35 минут (сложная архитектура)")
#     print("🎯 Ожидается accuracy 50-60% и F1-score 0.45-0.55")
#     print("📊 Мониторим F1-macro как главную метрику")
#     print("")

#     enhanced_model_v2, enhanced_results_v2, feature_categories_v2, features_v2 = train_enhanced_forex_model_v2(forex_data)
# else:
#     print("⚠️ Forex данные не загружены. Выполните ячейку загрузки данных!")

# print("\n🚀 ЧТО НОВОГО В V2:")
# print("✅ EnhancedForexLSTMModel класс")
# print("✅ F1-macro как основная метрика")
# print("✅ Precision/Recall для каждого класса")
# print("✅ Анализ уверенности модели")
# print("✅ Автосохранение scaler и модели")
# print("✅ Расширенная визуализация (12 графиков)")
# print("✅ Детальная сводка результатов")
# print("✅ Практические рекомендации")

In [99]:
!pip install catboost shap

In [100]:
# ЗАГРУЗКА СОХРАНЕННОЙ LSTM МОДЕЛИ ДЛЯ АНСАМБЛЯ
# ==================================================================================

import pickle
import os
import tensorflow as tf

def load_saved_model_and_prepare_ensemble():
    """
    Загрузка сохраненной LSTM модели и подготовка данных для ансамбля
    """
    print("🔄 ЗАГРУЖАЕМ СОХРАНЕННУЮ МОДЕЛЬ...")
    print("="*50)

    # 1. Проверяем существование файлов
    model_path = 'models/best_enhanced_model.h5'
    scaler_path = 'models/scaler.pkl'

    if not os.path.exists(model_path):
        print(f"❌ Модель не найдена: {model_path}")
        return None

    if not os.path.exists(scaler_path):
        print(f"❌ Scaler не найден: {scaler_path}")
        return None

    # 2. Загружаем LSTM модель
    print("📂 Загрузка LSTM модели...")
    try:
        # Кастомные метрики нужно определить для загрузки
        def f1_macro(y_true, y_pred):
            def f1_score_class(class_id):
                y_true_class = tf.cast(tf.equal(tf.argmax(y_true, axis=1), class_id), tf.float32)
                y_pred_class = tf.cast(tf.equal(tf.argmax(y_pred, axis=1), class_id), tf.float32)

                tp = tf.reduce_sum(y_true_class * y_pred_class)
                fp = tf.reduce_sum((1 - y_true_class) * y_pred_class)
                fn = tf.reduce_sum(y_true_class * (1 - y_pred_class))

                precision = tp / (tp + fp + tf.keras.backend.epsilon())
                recall = tp / (tp + fn + tf.keras.backend.epsilon())

                return 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())

            f1_0 = f1_score_class(0)
            f1_1 = f1_score_class(1)
            f1_2 = f1_score_class(2)

            return (f1_0 + f1_1 + f1_2) / 3

        def precision_class(class_id):
            def precision(y_true, y_pred):
                y_true_class = tf.cast(tf.equal(tf.argmax(y_true, axis=1), class_id), tf.float32)
                y_pred_class = tf.cast(tf.equal(tf.argmax(y_pred, axis=1), class_id), tf.float32)

                tp = tf.reduce_sum(y_true_class * y_pred_class)
                fp = tf.reduce_sum((1 - y_true_class) * y_pred_class)

                return tp / (tp + fp + tf.keras.backend.epsilon())
            return precision

        def recall_class(class_id):
            def recall(y_true, y_pred):
                y_true_class = tf.cast(tf.equal(tf.argmax(y_true, axis=1), class_id), tf.float32)
                y_pred_class = tf.cast(tf.equal(tf.argmax(y_pred, axis=1), class_id), tf.float32)

                tp = tf.reduce_sum(y_true_class * y_pred_class)
                fn = tf.reduce_sum(y_true_class * (1 - y_pred_class))

                return tp / (tp + fn + tf.keras.backend.epsilon())
            return recall

        # Загружаем с кастомными метриками
        custom_objects = {
            'f1_macro': f1_macro,
            'precision': precision_class(0),
            'precision_1': precision_class(1),
            'precision_2': precision_class(2),
            'recall': recall_class(0),
            'recall_1': recall_class(1),
            'recall_2': recall_class(2)
        }

        lstm_model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
        print("✅ LSTM модель загружена!")

    except Exception as e:
        print(f"❌ Ошибка загрузки модели: {e}")
        return None

    # 3. Загружаем scaler
    print("📂 Загрузка scaler...")
    try:
        with open(scaler_path, 'rb') as f:
            scaler = pickle.load(f)
        print("✅ Scaler загружен!")
    except Exception as e:
        print(f"❌ Ошибка загрузки scaler: {e}")
        return None

    # 4. Воссоздаем данные (должны быть доступны)
    if 'forex_data' not in globals():
        print("❌ forex_data не найден! Запустите ячейки 1-5!")
        return None

    print("🔧 Воссоздание данных...")

    # Пересоздаем все данные
    enhanced_data = calculate_enhanced_technical_indicators(forex_data)
    final_data = create_target_variable(enhanced_data)
    X, y, features, categories = prepare_enhanced_features_for_lstm(final_data)

    print(f"✅ Данные готовы: X{X.shape}, y{y.shape}")
    print(f"📊 Признаков: {len(features)}")

    # 5. Создаем объект модели
    class LoadedEnhancedModel:
        def __init__(self, model, scaler):
            self.model = model
            self.scaler = scaler
            self.sequence_length = 96  # Из настроек

        def predict(self, X_sequences):
            return self.model.predict(X_sequences, verbose=0)

        def prepare_data(self, X, y):
            """Подготовка данных как в оригинальной модели"""
            print("🔧 Подготовка данных...")

            # Нормализация
            X_scaled = self.scaler.transform(X)

            # Создание последовательностей
            X_sequences = []
            y_sequences = []

            for i in range(self.sequence_length, len(X_scaled)):
                X_sequences.append(X_scaled[i-self.sequence_length:i])
                y_sequences.append(y[i])

            X_sequences = np.array(X_sequences, dtype=np.float32)
            y_sequences = np.array(y_sequences)

            # Разделение данных
            total_samples = len(X_sequences)
            test_size = int(total_samples * 0.15)
            val_size = int(total_samples * 0.15)
            train_size = total_samples - test_size - val_size

            X_train = X_sequences[:train_size]
            y_train = y_sequences[:train_size]
            X_val = X_sequences[train_size:train_size + val_size]
            y_val = y_sequences[train_size:train_size + val_size]
            X_test = X_sequences[train_size + val_size:]
            y_test = y_sequences[train_size + val_size:]

            # Конвертация в категории
            def to_categorical_custom(y_data):
                y_cat = y_data + 1
                return tf.keras.utils.to_categorical(y_cat, num_classes=3)

            y_train_cat = to_categorical_custom(y_train)
            y_val_cat = to_categorical_custom(y_val)
            y_test_cat = to_categorical_custom(y_test)

            print(f"📊 Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

            return (X_train, y_train_cat), (X_val, y_val_cat), (X_test, y_test_cat)

    # 6. Создаем объект загруженной модели
    enhanced_model_v2 = LoadedEnhancedModel(lstm_model, scaler)

    print("\n✅ ВСЕ ГОТОВО!")
    print("📊 Переменные созданы:")
    print("   • enhanced_model_v2 (загруженная LSTM)")
    print("   • X, y (данные)")
    print("   • features_v2 (список признаков)")
    print("   • categories (категории признаков)")

    return enhanced_model_v2, X, y, features, categories

# ВЫПОЛНЯЕМ ЗАГРУЗКУ
print("🚀 ЗАГРУЖАЕМ СОХРАНЕННУЮ МОДЕЛЬ...")
result = load_saved_model_and_prepare_ensemble()

if result is not None:
    enhanced_model_v2, X, y, features_v2, categories = result

    print("\n🔥 ГОТОВО К ЗАПУСКУ АНСАМБЛЯ!")
    print("💡 Теперь можете запустить:")
    print("   ensemble_model, ensemble_results, feature_importance = train_lstm_catboost_ensemble()")
else:
    print("❌ Не удалось загрузить модель!")

🚀 ЗАГРУЖАЕМ СОХРАНЕННУЮ МОДЕЛЬ...
🔄 ЗАГРУЖАЕМ СОХРАНЕННУЮ МОДЕЛЬ...
📂 Загрузка LSTM модели...


✅ LSTM модель загружена!
📂 Загрузка scaler...
✅ Scaler загружен!
🔧 Воссоздание данных...
🔧 Расчет улучшенных технических индикаторов...
✅ Рассчитано 66 индикаторов
📊 Добавлено из документа: ADX, Williams %R, CCI, Stochastic
🔥 Новые фичи: взаимодействия, кластеры, сессии
🎯 Создание целевой переменной (прогноз на 60 минут)...
📊 Распределение целевой переменной:
   SELL: 1,856 (32.8%)
   HOLD: 1,710 (30.2%)
   BUY: 2,087 (36.9%)
🔧 Подготовка улучшенных признаков для LSTM...
✅ Подготовлено признаков: 63 (было 19)
📊 Чистых образцов: 5,587

📋 Признаки по категориям:
   Основные: 6 (Close, High, Low...)
   Технические: 12 (SMA_9, SMA_21, EMA_12...)
   Продвинутые: 13 (ADX, DI_Plus, DI_Minus...)
   Временные: 5 (Hour, DayOfWeek, IsLondonSession...)
   Сигналы: 11 (MA_Cross, EMA_Cross, MACD_Cross...)
   Лаги: 10 (Returns_lag_1, Returns_lag_2, Returns_lag_3...)
   Взаимодействия: 4 (RSI_MACD_Interaction, BB_RSI_Interaction, ADX_RSI_Strength...)
   Кластеры: 2 (Bullish_Cluster, Bearish_Cluster)



In [101]:
# ПЕРЕМЕЩАЕМ ФАЙЛ И СОЗДАЕМ НУЖНЫЕ ПЕРЕМЕННЫЕ
import shutil
import os

print("🔧 ИСПРАВЛЯЕМ ПУТИ ФАЙЛОВ...")

# Создаем папку models
os.makedirs('models', exist_ok=True)

# Перемещаем файл модели
if os.path.exists('best_model.h5'):
    shutil.move('best_model.h5', 'models/best_enhanced_model.h5')
    print("✅ Модель перемещена в models/best_enhanced_model.h5")
else:
    print("❌ best_model.h5 не найден в корне")

# Проверяем что есть scaler в памяти и сохраняем его
import pickle

# Ищем scaler в переменных
scaler_found = False
for var_name in ['enhanced_model_v2', 'trained_model']:
    if var_name in globals():
        var = globals()[var_name]
        if hasattr(var, 'scaler'):
            print(f"✅ Найден scaler в {var_name}")
            with open('models/scaler.pkl', 'wb') as f:
                pickle.dump(var.scaler, f)
            print("✅ Scaler сохранен в models/scaler.pkl")
            scaler_found = True
            break

if not scaler_found:
    print("⚠️ Scaler не найден, создаем новый...")
    # Пересоздаем scaler
    from sklearn.preprocessing import RobustScaler
    enhanced_data = calculate_enhanced_technical_indicators(forex_data)
    final_data = create_target_variable(enhanced_data)
    X, y, features_v2, categories = prepare_enhanced_features_for_lstm(final_data)

    scaler = RobustScaler()
    scaler.fit(X.values)

    with open('models/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    print("✅ Новый scaler создан и сохранен!")

# Проверяем результат
model_exists = os.path.exists('models/best_enhanced_model.h5')
scaler_exists = os.path.exists('models/scaler.pkl')

print(f"\n📊 РЕЗУЛЬТАТ:")
print(f"📁 Модель: {model_exists}")
print(f"📁 Scaler: {scaler_exists}")

if model_exists and scaler_exists:
    print("🔥 ВСЕ ГОТОВО! Теперь запустите:")
    print("result = load_saved_model_and_prepare_ensemble()")
    print("enhanced_model_v2, X, y, features_v2, categories = result")
else:
    print("❌ Что-то пошло не так...")

🔧 ИСПРАВЛЯЕМ ПУТИ ФАЙЛОВ...
❌ best_model.h5 не найден в корне
✅ Найден scaler в enhanced_model_v2
✅ Scaler сохранен в models/scaler.pkl

📊 РЕЗУЛЬТАТ:
📁 Модель: True
📁 Scaler: True
🔥 ВСЕ ГОТОВО! Теперь запустите:
result = load_saved_model_and_prepare_ensemble()
enhanced_model_v2, X, y, features_v2, categories = result


In [102]:
# ТОЛЬКО CATBOOST МОДЕЛЬ (БЕЗ АНСАМБЛЯ С LSTM)
# ==================================================================================

from catboost import CatBoostClassifier
import shap
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

def train_catboost_standalone():
    """
    Обучение только CatBoost модели на новых данных (63 признака)
    """
    print("🚀 ОБУЧЕНИЕ CATBOOST МОДЕЛИ (STANDALONE)")
    print("="*60)
    print("💡 Обходим проблему совместимости с LSTM")

    # 1. Подготавливаем данные
    print("🔧 Подготовка данных...")
    enhanced_data = calculate_enhanced_technical_indicators(forex_data)
    final_data = create_target_variable(enhanced_data)
    X, y, features, categories = prepare_enhanced_features_for_lstm(final_data)

    print(f"📊 Данные: X{X.shape}, y{y.shape}")
    print(f"🔢 Признаков: {len(features)}")

    # 2. Нормализация данных
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X.values)

    # 3. Разделение данных (как у LSTM)
    total_samples = len(X_scaled)
    test_size = int(total_samples * 0.15)
    val_size = int(total_samples * 0.15)
    train_size = total_samples - test_size - val_size

    X_train = X_scaled[:train_size]
    y_train = y[:train_size]
    X_val = X_scaled[train_size:train_size + val_size]
    y_val = y[train_size:train_size + val_size]
    X_test = X_scaled[train_size + val_size:]
    y_test = y[train_size + val_size:]

    print(f"📊 Разделение данных:")
    print(f"   Train: {len(X_train)}")
    print(f"   Val: {len(X_val)}")
    print(f"   Test: {len(X_test)}")

    # 4. Обучение CatBoost
    print("\n🚀 Обучение CatBoost модели...")

    catboost_model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.1,
        depth=6,
        l2_leaf_reg=3,
        random_seed=42,
        verbose=100,
        early_stopping_rounds=50,
        eval_metric='MultiClass',
        custom_metric=['F1'],
        auto_class_weights='Balanced',
        bootstrap_type='Bernoulli',
        subsample=0.8,
        colsample_bylevel=0.8,
        use_best_model=True
    )

    # Обучение с валидацией
    catboost_model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        plot=False
    )

    print("✅ CatBoost обучен!")

    # 5. Важность признаков
    feature_importance = catboost_model.get_feature_importance()
    feature_importance_df = pd.DataFrame({
        'feature': features,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)

    print(f"\n📊 ТОП-15 важных признаков:")
    for i, row in feature_importance_df.head(15).iterrows():
        print(f"   {row['feature']}: {row['importance']:.1f}")

    # 6. Предсказания
    print("\n🤖 Создание предсказаний...")
    y_pred_proba = catboost_model.predict_proba(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)

    # 7. Оценка
    accuracy = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')

    print(f"\n🎯 РЕЗУЛЬТАТЫ CATBOOST:")
    print(f"   Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   F1-macro: {f1_macro:.4f}")

    # Детальный отчет
    class_names = ['SELL', 'HOLD', 'BUY']
    print(f"\n📋 Детальный отчет:")
    print(classification_report(y_test, y_pred, target_names=class_names))

    # 8. SHAP анализ
    print("\n🔍 SHAP анализ...")
    try:
        # Берем небольшую выборку для SHAP
        sample_size = min(100, len(X_test))
        X_sample = X_test[:sample_size]

        explainer = shap.TreeExplainer(catboost_model)
        shap_values = explainer.shap_values(X_sample)

        print(f"✅ SHAP анализ завершен для {sample_size} образцов")

    except Exception as e:
        print(f"⚠️ SHAP анализ не удался: {e}")
        explainer, shap_values = None, None

    # 9. Сохранение
    print("\n💾 Сохранение модели...")
    catboost_model.save_model('models/catboost_standalone.cbm')
    joblib.dump(scaler, 'models/catboost_scaler.pkl')
    joblib.dump(feature_importance_df, 'models/feature_importance.pkl')

    # 10. Визуализация
    create_catboost_visualization(
        catboost_model, y_test, y_pred, y_pred_proba,
        feature_importance_df, categories, features
    )

    results = {
        'model': catboost_model,
        'scaler': scaler,
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'predictions': y_pred,
        'probabilities': y_pred_proba,
        'true_labels': y_test,
        'feature_importance': feature_importance_df,
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, target_names=class_names, output_dict=True)
    }

    return results

def create_catboost_visualization(model, y_true, y_pred, y_pred_proba, feature_importance_df, categories, features):
    """Визуализация результатов CatBoost"""

    plt.figure(figsize=(20, 12))

    # 1. Confusion Matrix
    plt.subplot(2, 4, 1)
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['SELL', 'HOLD', 'BUY'],
                yticklabels=['SELL', 'HOLD', 'BUY'])
    plt.title('🎯 Confusion Matrix', fontweight='bold')

    # 2. Feature Importance (Top 15)
    plt.subplot(2, 4, 2)
    top_features = feature_importance_df.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), [f[:10] for f in top_features['feature']], fontsize=8)
    plt.title('📊 Top 15 Features', fontweight='bold')
    plt.xlabel('Importance')

    # 3. Predictions Distribution
    plt.subplot(2, 4, 3)
    unique, counts = np.unique(y_pred, return_counts=True)
    class_names = ['SELL', 'HOLD', 'BUY']
    pred_counts = [counts[unique == i][0] if i in unique else 0 for i in range(3)]
    plt.pie(pred_counts, labels=class_names, autopct='%1.1f%%',
            colors=['red', 'gray', 'green'], startangle=90)
    plt.title('🎯 Predictions', fontweight='bold')

    # 4. Model Confidence
    plt.subplot(2, 4, 4)
    confidence = np.max(y_pred_proba, axis=1)
    plt.hist(confidence, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(confidence.mean(), color='red', linestyle='--',
                label=f'Mean: {confidence.mean():.3f}')
    plt.title('📊 Model Confidence', fontweight='bold')
    plt.xlabel('Max Probability')
    plt.ylabel('Count')
    plt.legend()

    # 5. Class Performance
    plt.subplot(2, 4, 5)
    from sklearn.metrics import classification_report
    class_report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

    classes = ['SELL', 'HOLD', 'BUY']
    precision_scores = [class_report[cls]['precision'] for cls in classes]
    recall_scores = [class_report[cls]['recall'] for cls in classes]

    x = np.arange(len(classes))
    width = 0.35
    plt.bar(x - width/2, precision_scores, width, label='Precision', alpha=0.7)
    plt.bar(x + width/2, recall_scores, width, label='Recall', alpha=0.7)
    plt.xlabel('Classes')
    plt.ylabel('Score')
    plt.title('⚖️ Precision vs Recall', fontweight='bold')
    plt.xticks(x, classes)
    plt.legend()

    # 6. Features by Category
    plt.subplot(2, 4, 6)
    category_counts = {k: len(v) for k, v in categories.items() if v}
    plt.pie(category_counts.values(), labels=list(category_counts.keys()),
            autopct='%1.0f%%', startangle=90)
    plt.title('🗂️ Features by Category', fontweight='bold')

    # 7. Accuracy by Class
    plt.subplot(2, 4, 7)
    f1_scores = [class_report[cls]['f1-score'] for cls in classes]
    bars = plt.bar(classes, f1_scores, color=['red', 'gray', 'green'], alpha=0.7)
    plt.title('📈 F1-Score by Class', fontweight='bold')
    plt.ylabel('F1-Score')
    for bar, score in zip(bars, f1_scores):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}', ha='center', va='bottom')

    # 8. Top Features by Category
    plt.subplot(2, 4, 8)
    # Группируем топ-10 признаков по категориям
    top_10_features = feature_importance_df.head(10)['feature'].tolist()
    category_importance = {}

    for category, cat_features in categories.items():
        if cat_features:
            importance_sum = 0
            for feature in cat_features:
                if feature in top_10_features:
                    idx = feature_importance_df[feature_importance_df['feature'] == feature].index[0]
                    importance_sum += feature_importance_df.loc[idx, 'importance']
            if importance_sum > 0:
                category_importance[category] = importance_sum

    if category_importance:
        cats = list(category_importance.keys())
        values = list(category_importance.values())
        plt.bar(range(len(cats)), values, alpha=0.7)
        plt.xticks(range(len(cats)), [c[:8] for c in cats], rotation=45)
        plt.title('📊 Top Categories', fontweight='bold')
        plt.ylabel('Importance Sum')

    plt.tight_layout()
    plt.show()

def print_catboost_summary(results):
    """Печать итоговой сводки CatBoost"""
    print("\n" + "="*60)
    print("🏆 ИТОГИ CATBOOST МОДЕЛИ")
    print("="*60)

    print(f"📊 ОСНОВНЫЕ РЕЗУЛЬТАТЫ:")
    print(f"   🎯 Accuracy: {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
    print(f"   🏆 F1-macro: {results['f1_macro']:.4f}")

    # Сравнение с LSTM
    lstm_accuracy = 0.3168  # Из прошлых результатов
    improvement = (results['accuracy'] - lstm_accuracy) / lstm_accuracy * 100

    print(f"\n📈 СРАВНЕНИЕ С LSTM:")
    print(f"   LSTM accuracy: {lstm_accuracy:.4f} (31.68%)")
    print(f"   CatBoost accuracy: {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
    print(f"   Улучшение: {improvement:+.1f}%")

    if improvement > 0:
        print(f"   ✅ CatBoost превосходит LSTM на {improvement:.1f}%!")
    else:
        print(f"   ⚠️ CatBoost немного хуже LSTM")

    print(f"\n💾 СОХРАНЕНО:")
    print(f"   📁 models/catboost_standalone.cbm")
    print(f"   📁 models/catboost_scaler.pkl")
    print(f"   📁 models/feature_importance.pkl")

    print("="*60)
    print("✅ CATBOOST МОДЕЛЬ ГОТОВА!")
    print("="*60)

# ЗАПУСК
print("🚀 ЗАПУСК CATBOOST STANDALONE МОДЕЛИ...")
catboost_results = train_catboost_standalone()
print_catboost_summary(catboost_results)

print("\n💡 МОДЕЛЬ ГОТОВА К ИСПОЛЬЗОВАНИЮ!")
print("🎯 CatBoost показал результаты на новых 63 признаках!")

🚀 ЗАПУСК CATBOOST STANDALONE МОДЕЛИ...
🚀 ОБУЧЕНИЕ CATBOOST МОДЕЛИ (STANDALONE)
💡 Обходим проблему совместимости с LSTM
🔧 Подготовка данных...
🔧 Расчет улучшенных технических индикаторов...
✅ Рассчитано 66 индикаторов
📊 Добавлено из документа: ADX, Williams %R, CCI, Stochastic
🔥 Новые фичи: взаимодействия, кластеры, сессии
🎯 Создание целевой переменной (прогноз на 60 минут)...
📊 Распределение целевой переменной:
   SELL: 1,856 (32.8%)
   HOLD: 1,710 (30.2%)
   BUY: 2,087 (36.9%)
🔧 Подготовка улучшенных признаков для LSTM...
✅ Подготовлено признаков: 63 (было 19)
📊 Чистых образцов: 5,587

📋 Признаки по категориям:
   Основные: 6 (Close, High, Low...)
   Технические: 12 (SMA_9, SMA_21, EMA_12...)
   Продвинутые: 13 (ADX, DI_Plus, DI_Minus...)
   Временные: 5 (Hour, DayOfWeek, IsLondonSession...)
   Сигналы: 11 (MA_Cross, EMA_Cross, MACD_Cross...)
   Лаги: 10 (Returns_lag_1, Returns_lag_2, Returns_lag_3...)
   Взаимодействия: 4 (RSI_MACD_Interaction, BB_RSI_Interaction, ADX_RSI_Strength...

ValueError: Number of classes, 4, does not match size of target_names, 3. Try specifying the labels parameter

In [103]:
# В конце нового кода есть:
catboost_results = train_catboost_standalone()  # ← НОВАЯ ФУНКЦИЯ!

🚀 ОБУЧЕНИЕ CATBOOST МОДЕЛИ (STANDALONE)
💡 Обходим проблему совместимости с LSTM
🔧 Подготовка данных...
🔧 Расчет улучшенных технических индикаторов...
✅ Рассчитано 66 индикаторов
📊 Добавлено из документа: ADX, Williams %R, CCI, Stochastic
🔥 Новые фичи: взаимодействия, кластеры, сессии
🎯 Создание целевой переменной (прогноз на 60 минут)...
📊 Распределение целевой переменной:
   SELL: 1,856 (32.8%)
   HOLD: 1,710 (30.2%)
   BUY: 2,087 (36.9%)
🔧 Подготовка улучшенных признаков для LSTM...
✅ Подготовлено признаков: 63 (было 19)
📊 Чистых образцов: 5,587

📋 Признаки по категориям:
   Основные: 6 (Close, High, Low...)
   Технические: 12 (SMA_9, SMA_21, EMA_12...)
   Продвинутые: 13 (ADX, DI_Plus, DI_Minus...)
   Временные: 5 (Hour, DayOfWeek, IsLondonSession...)
   Сигналы: 11 (MA_Cross, EMA_Cross, MACD_Cross...)
   Лаги: 10 (Returns_lag_1, Returns_lag_2, Returns_lag_3...)
   Взаимодействия: 4 (RSI_MACD_Interaction, BB_RSI_Interaction, ADX_RSI_Strength...)
   Кластеры: 2 (Bullish_Cluster, Bear

ValueError: Number of classes, 4, does not match size of target_names, 3. Try specifying the labels parameter